In [40]:
import odbc
import win32com.client
import pandas as pd
import pyodbc

def connection():
    server = 'tcp:192.168.1.6'
    database = 'planeamiento'
    username = 'sa'
    password = 'sa'

    return pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)


In [41]:
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel where end_processing_date is null'

df = pd.read_sql(query,cnxn)

In [42]:
df

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None
...,...,...,...,...,...,...,...,...
34654,34695,Z:\Varios\,INTERNOS ACTUALIZADOS 2019.xlsx,2020-01-23 13:18:48.013,NaT,None,None,None
34655,34696,Z:\Varios\,stock en negativos 27 de septiembre.xlsx,2020-01-23 13:18:48.083,NaT,None,None,None
34656,34697,Z:\Ventas\,Libro1.xlsx,2020-01-23 13:18:48.230,NaT,None,None,None
34657,34698,Z:\Ventas\,presentacion 3-6 v2.xlsm,2020-01-23 13:18:48.250,NaT,None,None,None


In [77]:
def check_excel(file):
    
    #xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
    #file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
    xl = win32com.client.Dispatch('Excel.Application')
    xl.DisplayAlerts = False
    try:
        
        #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,Editable=False)
        wb = xl.Workbooks.Open(file, None, True)
    except Exception as e:
        #print(e)
        #xl.Application.Quit()
        return e , []
        
    datos = []
    try:
        for x in wb.Connections:
            for i in x.Ranges:
                try:
                    #PRIMER DATO QUE TRAE LA PRIMER CELDA
                    celda = i.Cells(1,1)
                    #LA COORDENADA DE LA PRIMER CELDA
                    coordenada = i.Cells(1,1).Address
                except Exception as p:
                    print(p)
                    
                try:
                    for p in i.ListObject.QueryTable.Parameters:
                        datos.append([x,
                                      x.ODBCConnection.CommandText,
                                      x.ODBCConnection.Connection,
                                      i.Parent.Name,
                                      celda,
                                      p.SourceRange.Address,
                                      p.SourceRange.Parent.Name,
                                      p.SourceRange.Value])

                except Exception as e:
                    try:
                        datos.append([x,
                                      x.ODBCConnection.CommandText,
                                      x.ODBCConnection.Connection,
                                      i.Parent.Name,
                                      celda,
                                      None,
                                      None,
                                      None])
                    except:
                        try:
                            for p in i.ListObject.QueryTable.Parameters:
                                datos.append([x,
                                              x.OLEDBConnection.CommandText,
                                              x.OLEDBConnection.Connection,
                                              i.Parent.Name,
                                              celda,
                                              p.SourceRange.Address,
                                              p.SourceRange.Parent.Name,
                                              p.SourceRange.Value])
                        except:
                            try:
                                datos.append([x,
                                              x.OLEDBConnection.CommandText,
                                              x.OLEDBConnection.Connection,
                                              i.Parent.Name,
                                              celda,
                                              None,
                                              None,
                                              None])
                            except:                            
                                datos.append([x,
                                          None,
                                          None,
                                          i.Parent.Name,
                                          celda,
                                          None,
                                          None,
                                          None])

    except Exception as c:
        return c , []
    

    #df = pd.DataFrame(datos)
    #df = df.applymap(str)
    #datos = map(str,datos)
    if datos == []:
        datos.append(['None',
                      'None',
                      'None',
                      'None',
                      'None',
                      'None',
                      'None',
                      'None'])
    elif type(datos[0]) == list:
        datos = [list(map(str,i))for i in datos]
    
    else:
        datos = list(map(str,datos))
#    xl.DisplayAlerts = False
    try:
        #wb.Close(0)
        wb.Close(False)
        #wb.Close(SaveChanges=0)
        #wb.Close(SaveChanges=0,Filename=file)
    except:
        wb.Close(SaveChanges = False)
        #wb.Close(SaveChanges = False,Filename=file)
    xl.Application.Quit()
    
    #return df
    return 0, datos

In [84]:
def insert_file_connection(EXCEL_ID,NAME,STR_CONN,SQL):
    
    aux_cnn = connection()
    
    cursor = aux_cnn.cursor()
    
    params = EXCEL_ID,NAME,STR_CONN,SQL
    
    #query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = %s, @name = '%s', @str_conn = '%s', @sql = '%s'"%params
    query = """EXEC planeamiento..ctrl_excel_InsertFileConnection  %s, '%s', '%s', '%s'"""%params
    #sql = "{CALL planeamiento..ctrl_excel_InsertFileConnection (?,?,?,?)}"
    
    try:
        #cursor.execute(sql, params)
        cursor.execute(query)
        val = cursor.fetchval()
        cursor.commit()
        
        
        
    except Exception as e:
        print('*'*100)
        print(e)
        
    
    aux_cnn.close()
    return int(val)
    
    
    

In [27]:
#x = insert_file_connection(5,'PROBANDO NAME','CONEXION PROBANDO','SELECT * FROM PRUEBA')

In [ ]:
x

In [ ]:
cursor = cnxn.cursor()
for index,row in df.iterrows():                                                          
    cursor.execute("INSERT INTO tabla(,,,]) values (?,?,?)",
                   row['col'])
    cnxn.commit()
cursor.close()
cnxn.close()


datos = []



In [1]:
#PROBANDO CON SQLALCHEMY
from sqlalchemy import create_engine
import urllib
server = 'tcp:192.168.1.6'
database = 'planeamiento'
username = 'sa'
password = 'sa'

params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER='+
                                 server+';DATABASE='+database+';UID='+username+';PWD='+ password)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [2]:
EXCEL_ID = 5
NAME ='PROBANDO NAME'
STR_CONN = 'CONEXION PROBANDO'
SQL = 'SELECT * FROM PRUEBA'

In [3]:
parametros = (EXCEL_ID,NAME,STR_CONN,SQL)
query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = %s, @name = '%s', @str_conn = '%s', @sql = '%s'"%parametros

In [5]:
import pandas as pd

In [6]:
#parametros = (EXCEL_ID,NAME,STR_CONN,SQL)
    
#query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = {0}, @name = {1}, @str_conn = {2}, @sql = {3}".format(params)
    
#query = 'EXEC my_procedure @Param = {0}'.format(my_param)

In [7]:
df = pd.read_sql_query(query, engine)
#*** HASTA SQLALCHEMY

In [8]:
df

,ctrl_excel_connection_id
0,111.0


In [15]:
PPP= df.loc[2]['path']+df.loc[2]['filename']

In [75]:
# 1 - creo un df con la query 'select * from planeamiento..ctrl_excel where end_processing_date is null'
# 2 - recorro el df leyendo cada excel en cada vuelta del loop llama a una función que devuelve todos los parametros (cadena de conexion,query, nombre de la query, parametros etc.)
# 3 - cada lista que devuelve la función la inserto en la base de datos. ( llamada a procedures)

# Estos son lo procedures:
#     planeamiento..ctrl_excel_InsertFileConnection (Devuelve un excel_connection_id para insertar en la proxima tabla)
#     planeamiento..ctrl_excel_InsertFileConnectionLocation (Devuelve un excel_connection_location_id para insertar en la prox tabla)
#     planeamiento..ctrl_excel_InsertFileConnectionParameter (Devuelve excel_connection_location_parameter_id)
    
    


In [53]:
import os
from glob import glob
archivos = []
PATH = "C:\\Users\\jhernandez\\Desktop"
EXT = ["*.xls","*.xlsx","*.xlsm"]
files = []
for x in EXT:
    a = [file
                     for path, subdir, files in os.walk(PATH)
                     for file in glob(os.path.join(path, x))]
    files.append(a)

files

[['C:\\Users\\jhernandez\\Desktop\\Calidad IMPRESION ROTULOS.xls',
  'C:\\Users\\jhernandez\\Desktop\\excels\\Calidad IMPRESION ROTULOS.xls'],
 ['C:\\Users\\jhernandez\\Desktop\\050220 AC scrapers.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\archivo de ejemplo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\~$archivo de ejemplo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\conteo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\ejemplo3.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\pruebaMod.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo_modFecha.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo - copia.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\carpeta3\\carpetata 4\\050220 AC scrapers.xlsx',
  'C:\\

In [46]:
archivos

['C:\\Users\\jhernandez\\Desktop\\Canjes electros.xlsm',
 'C:\\Users\\jhernandez\\Desktop\\excels\\macro 2.xlsm',
 'C:\\Users\\jhernandez\\Desktop\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\~$archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\conteo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\ejemplo3.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\pruebaMod.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo_modFecha.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo - copia.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\carpeta3\\carpetata 4\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\excels\\130120

In [47]:
import os
from glob import glob
archivos = []
PATH = "C:\\Users\\jhernandez\\Desktop"
EXT = "*.xls"

xls = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
EXT = "*.xlsx"
xlsx = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]

EXT = "*.xlsm"
xlsm = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]

archivos = xls + xlsx +xlsm

In [48]:
list(archivos)

['C:\\Users\\jhernandez\\Desktop\\Calidad IMPRESION ROTULOS.xls',
 'C:\\Users\\jhernandez\\Desktop\\excels\\Calidad IMPRESION ROTULOS.xls',
 'C:\\Users\\jhernandez\\Desktop\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\~$archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\conteo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\ejemplo3.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\pruebaMod.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo_modFecha.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo - copia.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\carpeta3\\carpetata 4\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernande

In [ ]:
for x in archivos:
    cont = 0
    #iteracion por cada x con check_excel
    # if check_excel(x)["parametros"] != null:
        #hacer un insert en: select * from planeamiento..ctrl_excel_connection_parameter 
            #ELSE: cont +1
    # if check_excel(x)["string_connection"]!= null:
        #hacer un insert en:select * from planeamiento..ctrl_excel_connection 
            #ELSE: cont +1
    #if check_excel(x)["text_command"]!= null:
        #hacer un insert en:select * from planeamiento..ctrl_excel_connection_location 
            #ELSE: cont +1
    #if cont > 0:
    #  insertar ese excel en una nueva tabla(excels candidatos a borrar) y describir que parametros le falta F:
    print(x)

In [80]:
import datetime
#datetime.datetime.today()

In [85]:
datos = []
errores = []
start = datetime.datetime.today()
for index,row in df.iterrows():
    
    archivo = row['path']+row['filename']
    result = check_excel(archivo)
    if result[0] != 0:
        
        errores.append([row['excel_id'],archivo,result[0]])
        
            
    else:
        respuesta = result[0]
        if result[1] != []:
            
            if type(result[1]) == list:
                for res in result[1]:
                    res.append(row['excel_id'])
                    res.append(index)
                
            else:
                result[1].append(row['excel_id'])
                result[1].append(index)
            datos += result[1]
            #raise
    if index == 100:
        
        end = datetime.datetime.today()
        print(end-start)
        raise
#     respuesta = result[0]
#     result[1]

AttributeError: Open.Close

In [82]:
index

66

In [71]:
#index

66

In [8]:
row

excel_id                                                              140
path                           Z:\Consultas\Catálogos\03- 2014\1403-1404\
filename                 Listado 1403-1404 versión 2 - con bombachas.xlsx
date_of_insertion                              2020-01-23 12:10:16.743000
start_processing_date                                                 NaT
end_processing_date                                                  None
process_id                                                           None
error_msg                                                            None
Name: 100, dtype: object

In [12]:
datos

[['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 40, 0],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 41, 1],
 ['Conexión',
  'SELECT taller.codigo, taller.nombre, taller.domicilio, taller.localidad, taller.telefono, taller.tarea\r\nFROM ESTIMACIONES.dbo.taller taller\r\nORDER BY taller.codigo',
  'ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES;UID=estimaciones;PWD=estimaciones;APP=2007 Microsoft Office \r\nsystem;WSID=WS245;DATABASE=ESTIMACIONES;LANGUAGE=us_english;Network=DBMSSO',
  'TALLERES',
  'codigo',
  'None',
  'None',
  'None',
  42,
  2],
 ['Conexión1',
  'SELECT tipos.codigo, tipos.nombre, tipos.categoria, tipos.modulo, tipos.tamanyo, tipos.taller\r\nFROM ESTIMACIONES.dbo.tipos tipos\r\nORDER BY tipos.codigo',
  'ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES;UID=estimaciones;PWD=estimaciones;APP=2007 Microsoft Office system;WSID=WS245;DATABASE=ESTIMACIONES;LANGUAGE=us_english;Network=DBMSSOCN',
  'TIPOS',
  'codigo',
  'None',
  'None'

In [101]:
len(errores)

0

In [102]:
index

24

In [191]:
end = datetime.datetime.today() #0:08:50.505707
print(end-start)

0:08:50.505707


In [103]:
len(datos)

40

In [8]:
len(errores) #23

40

In [86]:
index

100

In [104]:
print(row['path']+row['filename'])

Z:\Consultas\estado importacion.xlsm


In [83]:
datos = []
errores = []
start = datetime.datetime.today()

    
archivo = row['path']+row['filename']
result = check_excel(archivo)
if result[0] != 0:

    errores.append([row['excel_id'],archivo,result[0]])


else:
    respuesta = result[0]
    if result[1] != []:

        if type(result[1]) == list:
            for res in result[1]:
                res.append(row['excel_id'])
                res.append(index)

        else:
            result[1].append(row['excel_id'])
            result[1].append(index)
        datos += result[1]
        #raise
if index == 100:

    end = datetime.datetime.today()
    print(end-start)
    raise
#     respuesta = result[0]
#     result[1]

TypeError: Quit() takes 1 positional argument but 2 were given

In [37]:
#**DEBUG
#xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
#file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
xl = win32com.client.Dispatch('Excel.Application')
file= row['path']+row['filename']
try:
    wb = xl.Workbooks.Open(file, None,True)
except Exception as e:
    print(e)
    

datos = []

In [36]:
for x in wb.Connections:
    for i in x.Ranges:
        print(i.Cells(1,1)[0])
        

_cp


In [18]:
print(row['path']+row['filename'])

Z:\Consultas\Catálogos\03- 2014\1403-1404\Listado 1403-1404 versión 2 - con bombachas.xlsx


In [19]:
Q=[]
for x in wb.Connections:
    for i in x.Ranges:
        try:
            celda = i.Cells(1,1)[2]
        except:
            celda = i.Cells(1,1)[0]
        try:
            for p in i.ListObject.QueryTable.Parameters:
                
                    
                Q.append([x,
                              x.ODBCConnection.CommandText,
                              x.ODBCConnection.Connection,
                              i.Parent.Name,
                              celda,
                              p.SourceRange.Address,
                              p.SourceRange.Parent.Name,
                              p.SourceRange.Value])

        except Exception as e:
            try:
                Q.append([x,
                              x.ODBCConnection.CommandText,
                              x.ODBCConnection.Connection,
                              i.Parent.Name,
                              celda,
                              None,
                              None,
                              None])
            except:
                try:
                    for p in i.ListObject.QueryTable.Parameters:
                        Q.append([x,
                                      x.OLEDBConnection.CommandText,
                                      x.OLEDBConnection.Connection,
                                      i.Parent.Name,
                                      celda,
                                      p.SourceRange.Address,
                                      p.SourceRange.Parent.Name,
                                      p.SourceRange.Value])
                except:
                    try:
                        Q.append([x,
                                      x.OLEDBConnection.CommandText,
                                      x.OLEDBConnection.Connection,
                                      i.Parent.Name,
                                      celda,
                                      None,
                                      None,
                                      None])
                    except:                            
                        Q.append([x,
                                  None,
                                  None,
                                  i.Parent.Name,
                                  i.Cells(1,1)[2],
                                  None,
                                  None,
                                  None])

IndexError: list index out of range

In [181]:
Q

[]

In [34]:
result

(0,
 [['Consulta desde ESTIMACIONES',
   'SELECT vFechasProcesos2.g, vFechasProcesos2.cp, vFechasProcesos2.zona, vFechasProcesos2.segmento, vFechasProcesos2.categoria, vFechasProcesos2.estadoSIS, vFechasProcesos2.sistemaSIS, vFechasProcesos2.fechaCalendario, vFechasProcesos2.fechaReal, vFechasProcesos2.UnidadesIngresadas, vFechasProcesos2.PedidosIngresados, vFechasProcesos2.fechaLiberada, vFechasProcesos2.fechaRetenida, vFechasProcesos2.fechaDesarme, vFechasProcesos2.estadoActual, vFechasProcesos2.despacho72Hs, vFechasProcesos2.despachadoReal, vFechasProcesos2.Retraso, vFechasProcesos2.tipoEnvio, vFechasProcesos2.fechaProceso, vFechasProcesos2.areaResponsable, vFechasProcesos2.estadoProceso, vFechasProcesos2.fechaUltimoIngreso, vFechasProcesos2.fechaBajada, vFechasProcesos2.UltimaActualizacion, vFechasProcesos2.ubicacion, vFechasProcesos2.prioridadfecha, vFechasProcesos2.prioridadtipo, vFechasProcesos2.ComentarioRegional, vFechasProcesos2.ComentarioJB,vFechasProcesos2.catalogos\r\nFROM

In [157]:
index

In [58]:
datos

[['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 40, 0],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 41, 1],
 ['Conexión',
  'SELECT taller.codigo, taller.nombre, taller.domicilio, taller.localidad, taller.telefono, taller.tarea\r\nFROM ESTIMACIONES.dbo.taller taller\r\nORDER BY taller.codigo',
  'ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES;UID=estimaciones;PWD=estimaciones;APP=2007 Microsoft Office \r\nsystem;WSID=WS245;DATABASE=ESTIMACIONES;LANGUAGE=us_english;Network=DBMSSO',
  'TALLERES',
  '2.0',
  'None',
  'None',
  'None',
  42,
  2],
 ['Conexión1',
  'SELECT tipos.codigo, tipos.nombre, tipos.categoria, tipos.modulo, tipos.tamanyo, tipos.taller\r\nFROM ESTIMACIONES.dbo.tipos tipos\r\nORDER BY tipos.codigo',
  'ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES;UID=estimaciones;PWD=estimaciones;APP=2007 Microsoft Office system;WSID=WS245;DATABASE=ESTIMACIONES;LANGUAGE=us_english;Network=DBMSSOCN',
  'TIPOS',
  '01',
  'None',
  'None',
  'No

In [49]:
errores

[[83,
  'Z:\\Consultas\\Calendarios\\Calendario  2020.xlsx',
  AttributeError('Excel.Application.Workbooks')],
 [106,
  'Z:\\Consultas\\Calendarios\\en desuso\\Circuitos Zonas.xlsx',
  pywintypes.com_error(-2147418111,
                       'La llamada fue rechazada por el destinatario.',
                       None,
                       None)],
 [108,
  'Z:\\Consultas\\Calendarios\\en desuso\\despacho cat electro.xlsx',
  AttributeError('Excel.Application.Workbooks')],
 [109,
  'Z:\\Consultas\\Calendarios\\en desuso\\Distribuidores.xlsx',
  AttributeError('Excel.Application.Workbooks')],
 [110,
  'Z:\\Consultas\\Calendarios\\en desuso\\FECHAS 2019 CORREGIDO.xlsx',
  AttributeError('Excel.Application.Workbooks')],
 [111,
  'Z:\\Consultas\\Calendarios\\en desuso\\HOJA DE RUTAS (2).xlsx',
  AttributeError('Excel.Application.Workbooks')],
 [112,
  'Z:\\Consultas\\Calendarios\\en desuso\\HOJA DE RUTAS.xlsx',
  AttributeError('Excel.Application.Workbooks')],
 [113,
  'Z:\\Consultas\\Cale

In [59]:
row['path']+row['filename']

'Z:\\Consultas\\Catálogos\\03- 2014\\1403-1404\\Listado 1403-1404 versión 2 - con bombachas.xlsx'

In [62]:
df[df['excel_id']]

['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 107, 67]

In [63]:
100-65

35

In [64]:
df.loc[65]

excel_id                                                 105
path                     Z:\Consultas\Calendarios\en desuso\
filename                           Calendario  2019 ult.xlsx
date_of_insertion                 2020-01-23 12:10:15.320000
start_processing_date                                    NaT
end_processing_date                                     None
process_id                                              None
error_msg                                               None
Name: 65, dtype: object

In [91]:
errores

[[106,
  'Z:\\Consultas\\Calendarios\\en desuso\\Circuitos Zonas.xlsx',
  pywintypes.com_error(-2147418111,
                       'La llamada fue rechazada por el destinatario.',
                       None,
                       None)]]

In [89]:
df[df['excel_id']==140]

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
100,140,Z:\Consultas\Catálogos\03- 2014\1403-1404\,Listado 1403-1404 versión 2 - con bombachas.xlsx,2020-01-23 12:10:16.743,NaT,None,None,None


In [68]:
df.loc[64:]

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
64,104,Z:\Consultas\Calendarios\en desuso\,Calendario 2019 DIEGO.xlsx,2020-01-23 12:10:15.310,NaT,None,None,None
65,105,Z:\Consultas\Calendarios\en desuso\,Calendario 2019 ult.xlsx,2020-01-23 12:10:15.320,NaT,None,None,None
66,106,Z:\Consultas\Calendarios\en desuso\,Circuitos Zonas.xlsx,2020-01-23 12:10:15.327,NaT,None,None,None
67,107,Z:\Consultas\Calendarios\en desuso\,Copia de seguridad de Calendario 2014 Delis.xlsx,2020-01-23 12:10:15.337,NaT,None,None,None
68,108,Z:\Consultas\Calendarios\en desuso\,despacho cat electro.xlsx,2020-01-23 12:10:15.343,NaT,None,None,None
...,...,...,...,...,...,...,...,...
34654,34695,Z:\Varios\,INTERNOS ACTUALIZADOS 2019.xlsx,2020-01-23 13:18:48.013,NaT,None,None,None
34655,34696,Z:\Varios\,stock en negativos 27 de septiembre.xlsx,2020-01-23 13:18:48.083,NaT,None,None,None
34656,34697,Z:\Ventas\,Libro1.xlsx,2020-01-23 13:18:48.230,NaT,None,None,None
34657,34698,Z:\Ventas\,presentacion 3-6 v2.xlsm,2020-01-23 13:18:48.250,NaT,None,None,None


In [88]:
datos[-1]

['Consulta desde ESTIMACIONES3',
 'select *\r\nfrom hdv_HistoricoDeVentas',
 'ODBC;DSN=ESTIMACIONES;Description=estimaciones;UID=sa;PWD=sa;APP=2007 Microsoft Office system;WSID=WS146;DATABASE=ESTIMACIONES;Trusted_Connection=Yes',
 'elasticidades',
 '_conDesc',
 'None',
 'None',
 'None',
 140,
 100]

In [45]:
file = df.loc[65]['path']+df.loc[65]['filename']

In [52]:
file = df.loc[65]['path']+df.loc[65]['filename']
xl = win32com.client.Dispatch('Excel.Application')
xl.DisplayAlerts = False
try:
    wb = xl.Workbooks.Open(file, None,True)
except Exception as e:
    #print(e)
    #xl.Application.Quit()
    print(e)

datos = []

for x in wb.Connections:
    for i in x.Ranges:
        try:
            celda = i.Cells(1,1)[2]
        except:
            celda = i.Cells(1,1)[0]
        raise

RuntimeError: No active exception to reraise

In [62]:
#PRIMER DATO QUE TRAE LA PRIMER CELDA
print(i.Cells(1,1))
#LA COORDENADA DE LA PRIMER CELDA
print(i.Cells(1,1).Address)

com_error: (-2146827864, 'OLE error 0x800a01a8', None, None)

In [65]:
zona
$a$1

com_error: (-2146827864, 'OLE error 0x800a01a8', None, None)

In [109]:
import odbc
import win32com.client
import pandas as pd
import pyodbc

def connection():
    server = 'tcp:192.168.1.6'
    database = 'planeamiento'
    username = 'sa'
    password = 'sa'

    return pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)


In [110]:
import odbc
import win32com.client
import pandas as pd
import pyodbc


In [111]:
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel where end_processing_date is null'

df = pd.read_sql(query,cnxn)

In [112]:
df

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None
...,...,...,...,...,...,...,...,...
34654,34695,Z:\Varios\,INTERNOS ACTUALIZADOS 2019.xlsx,2020-01-23 13:18:48.013,NaT,None,None,None
34655,34696,Z:\Varios\,stock en negativos 27 de septiembre.xlsx,2020-01-23 13:18:48.083,NaT,None,None,None
34656,34697,Z:\Ventas\,Libro1.xlsx,2020-01-23 13:18:48.230,NaT,None,None,None
34657,34698,Z:\Ventas\,presentacion 3-6 v2.xlsm,2020-01-23 13:18:48.250,NaT,None,None,None


In [113]:
def check_excel(file):
    
    #xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
    #file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
    try:
        xl = win32com.client.Dispatch('Excel.Application')
        xl.DisplayAlerts = False
        try:
            #wb = xl.Workbooks.Open(file, None,True)
            #wb = xl.Workbooks.Open(Filename=file,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,ReadOnly=True)
            wb = xl.Workbooks.Open(file, None, True)

        except Exception as e:
            #print(e)
            #xl.Application.Quit()
            return e , []

        datos = []
        try:
            for x in wb.Connections:
                for i in x.Ranges:
                    try:
                        #PRIMER DATO QUE TRAE LA PRIMER CELDA
                        celda = i.Cells(1,1)
                        #LA COORDENADA DE LA PRIMER CELDA
                        coordenada = i.Cells(1,1).Address
                    except Exception as p:
                        print(p)

                    try:
                        for p in i.ListObject.QueryTable.Parameters:
                            datos.append([x,
                                          x.ODBCConnection.CommandText,
                                          x.ODBCConnection.Connection,
                                          i.Parent.Name,
                                          celda,
                                          p.SourceRange.Address,
                                          p.SourceRange.Parent.Name,
                                          p.SourceRange.Value])

                    except Exception as e:
                        try:
                            datos.append([x,
                                          x.ODBCConnection.CommandText,
                                          x.ODBCConnection.Connection,
                                          i.Parent.Name,
                                          celda,
                                          None,
                                          None,
                                          None])
                        except:
                            try:
                                for p in i.ListObject.QueryTable.Parameters:
                                    datos.append([x,
                                                  x.OLEDBConnection.CommandText,
                                                  x.OLEDBConnection.Connection,
                                                  i.Parent.Name,
                                                  celda,
                                                  p.SourceRange.Address,
                                                  p.SourceRange.Parent.Name,
                                                  p.SourceRange.Value])
                            except:
                                try:
                                    datos.append([x,
                                                  x.OLEDBConnection.CommandText,
                                                  x.OLEDBConnection.Connection,
                                                  i.Parent.Name,
                                                  celda,
                                                  None,
                                                  None,
                                                  None])
                                except:                            
                                    datos.append([x,
                                              None,
                                              None,
                                              i.Parent.Name,
                                              celda,
                                              None,
                                              None,
                                              None])

        except Exception as c:
            return c , []


        #df = pd.DataFrame(datos)
        #df = df.applymap(str)
        #datos = map(str,datos)
        if datos == []:
            datos.append(['None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None'])
        elif type(datos[0]) == list:
            datos = [list(map(str,i))for i in datos]

        else:
            datos = list(map(str,datos))
    #    xl.DisplayAlerts = False
        try:
            #wb.Close(0)
            #wb.Close(False)
            #wb.Close(SaveChanges=0)
            xl.ActiveWorkbook.Saved = True
            xl.ActiveWorkbook.Close
            #wb.Close(SaveChanges=0,Filename=file)
        except:
            wb.Close(SaveChanges = False)
            #wb.Close(SaveChanges = False,Filename=file)
        xl.Application.Quit()

        #return df
        return 0, datos
    except:
        return 'No se pudo abrir .xlsx',[]

In [114]:
import datetime
#datetime.datetime.today()

In [115]:
datos = []
errores = []
start = datetime.datetime.today()
for index,row in df.iterrows():
    
    archivo = row['path']+row['filename']
    #matar proceso excel
    result = check_excel(archivo)
    if result[0] != 0:
        
        errores.append([row['excel_id'],archivo,result[0],index])
        
            
    else:
        respuesta = result[0]
        if result[1] != []:
            
            if type(result[1]) == list:
                for res in result[1]:
                    res.append(row['excel_id'])
                    res.append(index)
                
            else:
                result[1].append(row['excel_id'])
                result[1].append(index)
            datos += result[1]
            #raise
    if index == 200:
        
        end = datetime.datetime.today()
        print(end-start)
        raise
#     respuesta = result[0]
#     result[1]

0:30:49.418623


RuntimeError: No active exception to reraise

In [119]:

print(end-start)

0:30:49.418623


In [116]:
len(datos) #162

330

In [117]:
len(errores) #2

2

In [118]:
index

200

In [124]:
errores

[[56,
  'Z:\\....Candidatos a borrar\\varios\\dortiz Planif\\BOTONERA\\LISTADO DE BOTONERA XX-10.xlsx',
  'No se pudo abrir .xlsx',
  16],
 [227,
  'Z:\\Consultas\\Catálogos\\03- 2014\\1409-1410\\Listado 1409-1410 versión 7.xlsm',
  'No se pudo abrir .xlsx',
  187]]

In [89]:
datos

[['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 40, 0],
 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 41, 1],
 ['Conexión',
  'SELECT taller.codigo, taller.nombre, taller.domicilio, taller.localidad, taller.telefono, taller.tarea\r\nFROM ESTIMACIONES.dbo.taller taller\r\nORDER BY taller.codigo',
  'ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES;UID=estimaciones;PWD=estimaciones;APP=2007 Microsoft Office \r\nsystem;WSID=WS245;DATABASE=ESTIMACIONES;LANGUAGE=us_english;Network=DBMSSO',
  'TALLERES',
  'codigo',
  'None',
  'None',
  'None',
  42,
  2],
 ['Conexión1',
  'SELECT tipos.codigo, tipos.nombre, tipos.categoria, tipos.modulo, tipos.tamanyo, tipos.taller\r\nFROM ESTIMACIONES.dbo.tipos tipos\r\nORDER BY tipos.codigo',
  'ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES;UID=estimaciones;PWD=estimaciones;APP=2007 Microsoft Office system;WSID=WS245;DATABASE=ESTIMACIONES;LANGUAGE=us_english;Network=DBMSSOCN',
  'TIPOS',
  'codigo',
  'None',
  'None'

In [149]:
result[1]

[['Consulta desde ESTIMACIONES3',
  'select *\r\nfrom hdv_HistoricoDeVentas',
  'ODBC;DSN=ESTIMACIONES;Description=estimaciones;UID=sa;PWD=sa;APP=2007 Microsoft Office system;WSID=WS146;DATABASE=ESTIMACIONES;Trusted_Connection=Yes',
  'elasticidades',
  '_cp',
  'None',
  'None',
  'None',
  140,
  100]]

In [127]:
df = pd.DataFrame(datos)
#df[df[8]]
#df.groupby(8).sum
sin_query = df[df[1]!='None']

In [31]:
sin_query[1].count()

103

In [32]:
c_query = df[df[1]=='None']

In [33]:
len(c_query)

61

In [34]:
len(sin_query[8].unique())

40

In [151]:
df

,0,1,2,3,4,5,6,7,8,9
0,None,None,None,None,None,None,None,None,40,0
1,None,None,None,None,None,None,None,None,41,1
2,Conexión,"SELECT taller.codigo, taller.nombre, taller.do...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TALLERES,codigo,None,None,None,42,2
3,Conexión1,"SELECT tipos.codigo, tipos.nombre, tipos.categ...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TIPOS,codigo,None,None,None,42,2
4,Conexión2,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,COLOR,codigo,None,None,None,42,2
...,...,...,...,...,...,...,...,...,...,...
159,None,None,None,None,None,None,None,None,136,96
160,Consulta desde ESTIMACIONES,select *\r\nfrom hdv_HistoricoDeVentas,ODBC;DSN=ESTIMACIONES;Description=estimaciones...,2013-05-17,Suma de pVentasBase,None,None,None,137,97
161,None,None,None,None,None,None,None,None,138,98
162,Consulta desde ESTIMACIONES3,select *\r\nfrom hdv_HistoricoDeVentas,ODBC;DSN=ESTIMACIONES;Description=estimaciones...,elasticidades,_cp,None,None,None,139,99


In [38]:
row['path']+row['filename'] #'Z:\\Consultas\\Catálogos\\03- 2014\\1403-1404\\Listado 1403-1404 versión 2 - con bombachas.xlsx'

KeyError: 'path'

In [ ]:
"NOMBRE DE MODELOS.xlsx"
"TIPOS Y SUBTIPOS.xlsx"

In [25]:
archivo = row['path']+row['filename']

In [26]:
archivo

'Z:\\Consultas\\Catálogos\\03- 2014\\1403-1404\\Listado 1403-1404 versión 2 - con bombachas.xlsx'

In [93]:
row

excel_id                                                              140
path                           Z:\Consultas\Catálogos\03- 2014\1403-1404\
filename                 Listado 1403-1404 versión 2 - con bombachas.xlsx
date_of_insertion                              2020-01-23 12:10:16.743000
start_processing_date                                                 NaT
end_processing_date                                                  None
process_id                                                           None
error_msg                                                            None
Name: 100, dtype: object

In [92]:
df[df['filename']=='TIPOS Y SUBTIPOS.xlsx']

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
90,130,Z:\Consultas\CARGA DE CATALOGOS\,TIPOS Y SUBTIPOS.xlsx,2020-01-23 12:10:16.263,NaT,None,None,None


In [50]:
df.loc[90]['path']+df.loc[90]['filename']

Z:\Consultas\CARGA DE CATALOGOS\TIPOS Y SUBTIPOS.xlsx


In [76]:
file = df.loc[90]['path']+df.loc[90]['filename']
xl = win32com.client.Dispatch('Excel.Application')
xl.DisplayAlerts = True
try:
    #wb = xl.Workbooks.Open(file, None,True)
    wb = xl.Workbooks.Open(file, None, True)
    #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,Editable=False,ReadOnly=True,Origin=True)
    
except Exception as e:
    #print(e)
    #xl.Application.Quit()
    print(e)

In [104]:
df

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None
...,...,...,...,...,...,...,...,...
34654,34695,Z:\Varios\,INTERNOS ACTUALIZADOS 2019.xlsx,2020-01-23 13:18:48.013,NaT,None,None,None
34655,34696,Z:\Varios\,stock en negativos 27 de septiembre.xlsx,2020-01-23 13:18:48.083,NaT,None,None,None
34656,34697,Z:\Ventas\,Libro1.xlsx,2020-01-23 13:18:48.230,NaT,None,None,None
34657,34698,Z:\Ventas\,presentacion 3-6 v2.xlsm,2020-01-23 13:18:48.250,NaT,None,None,None


In [106]:
df[df["filename"] == "Faltannu por fecha.xls"]

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None


In [125]:
df = pd.DataFrame(datos)

In [126]:
df

,0,1,2,3,4,5,6,7,8,9
0,None,None,None,None,None,None,None,None,40,0
1,None,None,None,None,None,None,None,None,41,1
2,Conexión,"SELECT taller.codigo, taller.nombre, taller.do...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TALLERES,codigo,None,None,None,42,2
3,Conexión1,"SELECT tipos.codigo, tipos.nombre, tipos.categ...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TIPOS,codigo,None,None,None,42,2
4,Conexión2,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,COLOR,codigo,None,None,None,42,2
...,...,...,...,...,...,...,...,...,...,...
325,Consulta desde ESTIMACIONES1,"select v.cp, substring(ep.codigo, 1, 4) code,\...",ODBC;DSN=ESTIMACIONES;UID=sa;PWD=sa;APP=2007 M...,Listado p Diseño,cp,None,None,None,239,199
326,Consulta desde ESTIMACIONES3,select *\r\nfrom hdv_HistoricoDeVentas,ODBC;DSN=ESTIMACIONES;Description=estimaciones...,elasticidades,_cp,None,None,None,239,199
327,Consulta desde ESTIMACIONES,select *\r\nfrom hdv_HistoricoDeVentas,ODBC;DSN=ESTIMACIONES;Description=estimaciones...,Tabla,cp_inicio,None,None,None,240,200
328,Consulta desde ESTIMACIONES1,"select v.cp, substring(ep.codigo, 1, 4) code,\...",ODBC;DSN=ESTIMACIONES;UID=sa;PWD=sa;APP=2007 M...,Listado p Diseño,cp,None,None,None,240,200


In [128]:
df

,0,1,2,3,4,5,6,7,8,9
0,None,None,None,None,None,None,None,None,40,0
1,None,None,None,None,None,None,None,None,41,1
2,Conexión,"SELECT taller.codigo, taller.nombre, taller.do...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TALLERES,codigo,None,None,None,42,2
3,Conexión1,"SELECT tipos.codigo, tipos.nombre, tipos.categ...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TIPOS,codigo,None,None,None,42,2
4,Conexión2,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,COLOR,codigo,None,None,None,42,2
...,...,...,...,...,...,...,...,...,...,...
325,Consulta desde ESTIMACIONES1,"select v.cp, substring(ep.codigo, 1, 4) code,\...",ODBC;DSN=ESTIMACIONES;UID=sa;PWD=sa;APP=2007 M...,Listado p Diseño,cp,None,None,None,239,199
326,Consulta desde ESTIMACIONES3,select *\r\nfrom hdv_HistoricoDeVentas,ODBC;DSN=ESTIMACIONES;Description=estimaciones...,elasticidades,_cp,None,None,None,239,199
327,Consulta desde ESTIMACIONES,select *\r\nfrom hdv_HistoricoDeVentas,ODBC;DSN=ESTIMACIONES;Description=estimaciones...,Tabla,cp_inicio,None,None,None,240,200
328,Consulta desde ESTIMACIONES1,"select v.cp, substring(ep.codigo, 1, 4) code,\...",ODBC;DSN=ESTIMACIONES;UID=sa;PWD=sa;APP=2007 M...,Listado p Diseño,cp,None,None,None,240,200
